In [1]:
import requests

In [2]:
import re

In [3]:
import csv

In [4]:
from bs4 import BeautifulSoup

In [15]:
urllist = list()

In [29]:
incre = 1
with open("rank.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    for i in range(1,22):
        params = {"page":i}
        r = requests.get("http://www.zinch.cn/ss/us",params = params)
        soup = BeautifulSoup(r.text,"html.parser")
        for tag in soup.find_all("h2"):
            if "class" in list(tag.attrs.keys()):
                if "properties-name-en" in tag['class']:
                    urllist.append(tag.findNext().attrs['href']);
                    if tag.findNext().text.find('(') == -1:
                        spamwriter.writerow([incre,tag.findNext().text])
                    else:
                        ind = tag.findNext().text.find('(')
                        spamwriter.writerow([incre,tag.findNext().text[0:ind-1]])
                    incre = incre+1
                        

In [64]:
logolist = list()

In [40]:
from urllib.request import urlretrieve

In [65]:
for i in range(1,22):
    params = {"page":i}
    r = requests.get("http://www.zinch.cn/ss/us",params = params)
    soup = BeautifulSoup(r.text,"html.parser")
    for tag in soup.find_all("img"):
        if "class" in list(tag.attrs.keys()):
            if "imagecache-75x75" in tag['class']:
                logolist.append(tag.attrs['src']);

In [81]:
## Crawling the school logo image
for i in range(0,len(urllist)):
    fileurl = "./schoollogo/"+urllist[i][1:]+".jpg"
    urlretrieve(logolist[i],fileurl);

In [88]:
r = requests.get("http://www.zinch.cn/princeton-university/major_grad")

In [139]:
with open("zinchinfo.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    for i in range(0,210):
        r=requests.get("http://www.zinch.cn"+urllist[i]+"/major_grad")
        if r.status_code == 200:
            soup = BeautifulSoup(r.text,"html.parser")
            ans = list();
            for tag in soup.find_all("tr"):
                ans.append(tag.findNext().findNext().text);
            spamwriter.writerow(ans);
        else:
            spamwriter.writerow([np.nan])
            

In [134]:
import pandas as pd

In [286]:
import numpy as np

In [294]:
info = pd.read_csv("./zinchinfo.csv",header=None)

In [295]:
info = info.iloc[:,0:5]

In [296]:
for i in range(0,210):
    toe = info.iloc[i,0]
    sat = info.iloc[i,1]
    rate = info.iloc[i,4]/info.iloc[i,3]
    if str(toe) == "nan":
        continue;
    elif re.search("[0-9]+",toe)!=None:
        info.iloc[i,0] = re.search("[0-9]+",toe).group(0)
    else:
        info.iloc[i,0] = np.nan
    if str(sat) == "nan":
        continue;
    elif re.search("[0-9]+ - [0-9]+",sat)!=None:
        info.iloc[i,1] = re.search("[0-9]+ - [0-9]+",sat).group(0)
    else:
        info.iloc[i,1] = np.nan
    info.iloc[i,3] = rate;
info = info.iloc[:,0:4]

In [299]:
info.to_json("zinchdata.json")

In [ ]:
##0:toefl, 1:sat, 2:accrate, 3:interate